# Create an interface with no strain matching

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Set slab parameters for the substrate and film in cell 2.1. (or use default).
1. Set interface parameters in cell 3.1. (or use default).
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create substrate and film slabs and select the terminations
1. Generate an interface by wrapping the film slab into the substrate slab. Optionally, the film slab can be scaled to match the substrate lattice.
1. Select the interface with the desired strain and visualize it

## Notes
1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare Environment
### 1.1. Set notebook parameters

In [ ]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False 
# Enable scaling of the film slab atomic coordinates to match the substrate lattice (preserve coordinates in crystal units).
ENABLE_FILM_SCALING = True
# Create slabs for the film and substrate when creating the interface, otherwise use provided materials directly.
CREATE_SLABS = True 

FILM_INDEX = 1
FILM_MILLER_INDICES = (0, 0, 1)
FILM_THICKNESS = 1  # in atomic layers
FILM_VACUUM = 0.0  # in angstroms
FILM_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
FILM_USE_ORTHOGONAL_Z = True

SUBSTRATE_INDEX = 0
SUBSTRATE_MILLER_INDICES = (1, 1, 1)
SUBSTRATE_THICKNESS = 3  # in atomic layers
SUBSTRATE_VACUUM = 3.0  # in angstroms
SUBSTRATE_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
SUBSTRATE_USE_ORTHOGONAL_Z = True

# Set the termination pair indices
TERMINATION_PAIR_INDEX = 0 # Will be overridden in interactive selection is used
INTERFACE_DISTANCE = 3.0  # in Angstrom
INTERFACE_VACUUM = 20.0  # in Angstrom


### 1.2. Install Packages
Note: the step executes only when using Pyodide. For other environments (e.g. Colab, JupyterLab), the packages should be installed via `pip install .` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb")

### 1.3. Get input materials and assign `substrate` and `film`
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
substrate = materials[SUBSTRATE_INDEX]
try: 
    film = materials[FILM_INDEX]
except IndexError:
    print("Please select a film material. Film is set to substrate.")
    film = substrate

### 1.4. Preview Substrate and Film

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

## 2. Configure slabs and select termination pair

### 2.1. Create Substrate and Layer Slabs
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.
Define the substrate slab cell that will be used as a base for the interface and the film slab cell that will be placed on top of the substrate slab.

In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab

film_slab_configuration = SlabConfiguration(
    bulk=film,
    miller_indices=FILM_MILLER_INDICES,
    thickness=FILM_THICKNESS, # in atomic layers
    vacuum=FILM_VACUUM, # in angstroms
    xy_supercell_matrix=FILM_XY_SUPERCELL_MATRIX,
    use_orthogonal_z=FILM_USE_ORTHOGONAL_Z
)

substrate_slab_configuration = SlabConfiguration(
    bulk=substrate,
    miller_indices=SUBSTRATE_MILLER_INDICES,
    thickness=SUBSTRATE_THICKNESS, # in atomic layers
    vacuum=SUBSTRATE_VACUUM, # in angstroms
    xy_supercell_matrix=SUBSTRATE_XY_SUPERCELL_MATRIX,
    use_orthogonal_z=SUBSTRATE_USE_ORTHOGONAL_Z
)

### 2.2. Get possible terminations for the slabs

In [ ]:
film_slab_terminations = get_terminations(film_slab_configuration)
substrate_slab_terminations = get_terminations(substrate_slab_configuration)

### 2.3. Visualize slabs for all possible terminations

In [ ]:
film_slabs = [create_slab(film_slab_configuration, termination) for termination in film_slab_terminations]
substrate_slabs = [create_slab(substrate_slab_configuration, termination) for termination in substrate_slab_terminations]

visualize([{"material":slab, "title": slab.metadata["build"]["termination"]} for slab in film_slabs ], repetitions=[3, 3, 1], rotation="-90x")
visualize([{"material":slab, "title": slab.metadata["build"]["termination"]} for slab in substrate_slabs ], repetitions=[3, 3, 1], rotation="-90x")  

### 2.4. Print terminations for the interface

In [ ]:
from itertools import product

termination_pairs = list(product(film_slab_terminations, substrate_slab_terminations))    
print("Termination Pairs (Film, Substrate)")
for idx, termination_pair in enumerate(termination_pairs):
    print(f"    {idx}: {termination_pair}")
    

### 2.5. Select termination pair for the interface

In [ ]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

termination_pair_index = TERMINATION_PAIR_INDEX

termination_pair = termination_pairs[termination_pair_index]
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        termination_pair = await ui_prompt_select_array_element_by_index_pyodide(termination_pairs, element_name="film/substrate termination pair")
    else:
        termination_pair = ui_prompt_select_array_element_by_index(termination_pairs, element_name="film/substrate termination pair")

## 3. Create an interface
### 3.1. Initialize the Interface Configuration
We use no strain matching for the interface construction. Substrate slab cell is used as a base and the film slab is placed on top of it with the specified distance.

In [ ]:
from mat3ra.made.tools.build.interface import InterfaceConfiguration, create_interface

film_termination, substrate_termination = termination_pair
interface_configuration = InterfaceConfiguration(
    film_configuration=film_slab_configuration,
    substrate_configuration=substrate_slab_configuration,
    film_termination=film_termination,
    substrate_termination=substrate_termination,
    interface_distance=INTERFACE_DISTANCE, # in Angstrom
    interface_vacuum=INTERFACE_VACUUM # in Angstrom
)

### 3.2. Scale the film slab to match the substrate lattice (optional)

In [ ]:
from mat3ra.made.tools.build.interface import SimpleInterfaceBuilder, SimpleInterfaceBuilderParameters

builder = SimpleInterfaceBuilder(build_parameters=SimpleInterfaceBuilderParameters(scale_film=ENABLE_FILM_SCALING, create_slabs=CREATE_SLABS))
interface = builder.get_material(configuration=interface_configuration)

## 4. Preview the selected material

In [ ]:
visualize([interface], repetitions=[3, 3, 1])
visualize([interface], repetitions=[3, 3, 1], rotation="-90x")

## 5. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials
set_materials(interface)